# **Skenario 4 - CNN DenseNet121**

Skenario 4 menggunakan model Convolutional Neural Network (CNN) dengan arsitektur DenseNet121 untuk melakukan klasifikasi pada data citra otak.

---

## 1. Import packages

Import library yang digunakan untuk preprocessing data, membangun model, dan visualisas

In [2]:
from glob import glob
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import backend as K
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
%matplotlib inline

## 2. Data Preparation

### 2.1 Load dan Eksplorasi Data
cari dan cetak data `selected_image.npz` di dalam struktur direktori dan menampilkan informasi tentang jumlah slice untuk setiap label dan plane

In [35]:
base_dir = r"D:\Users\RESA\Coding\Alzheimer-Classification-with-CNN-SVM\Notebook\Preprocessing\selected_images.npz"

# Memuat data dari file npz yang berbeda
loaded_data = np.load(base_dir, allow_pickle=True)

# Mendapatkan data dari objek array di dalam file .npz
loaded_combined_slices = loaded_data[list(loaded_data.keys())[0]]

# Contoh cara mengakses data yang dimuat
print("Data yang dimuat:")
for label, plane_slices in loaded_combined_slices.item().items():
    total_slices = len(plane_slices)
    print(f"Label: {label}, Jumlah Total Slice: {total_slices}")

Data yang dimuat:
Label: coronal, Jumlah Total Slice: 600
Label: axial, Jumlah Total Slice: 600
Label: sagittal, Jumlah Total Slice: 600


### 2.2 Pembagian Data

Menyiapkan data dengan menggabungkan slice dan melakukan label encoding.

In [25]:
labels = list(loaded_combined_slices.item().keys())

In [27]:
import shutil
from sklearn.model_selection import StratifiedKFold

n_splits = 5
stratified_kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(stratified_kfold.split(labels, labels)):
    train_data = {label: loaded_combined_slices.item()[label] for label in labels[train_index]}
    test_data = {label: loaded_combined_slices.item()[label] for label in labels[test_index]}

    train_dir = os.path.join(base_dir, f'train_fold_{fold+1}')
    test_dir = os.path.join(base_dir, f'test_fold_{fold+1}')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    # Save slices to training directory
    for label, plane_slices in train_data.items():
        label_dir = os.path.join(train_dir, label)
        os.makedirs(label_dir, exist_ok=True)

        for idx, slice_data in enumerate(plane_slices):
            slice_path = os.path.join(label_dir, f'slice_{idx}.npy')
            np.save(slice_path, slice_data)

    # Save slices to testing directory
    for label, plane_slices in test_data.items():
        label_dir = os.path.join(test_dir, label)
        os.makedirs(label_dir, exist_ok=True)

        for idx, slice_data in enumerate(plane_slices):
            slice_path = os.path.join(label_dir, f'slice_{idx}.npy')
            np.save(slice_path, slice_data)

    print(f'Fold {fold+1}: Total slices train - {sum(len(os.listdir(os.path.join(train_dir, label))) for label in labels)}, Total slices test - {sum(len(os.listdir(os.path.join(test_dir, label))) for label in labels)}')

ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=3.